In [ ]:
import os
import re
import sys
import h5py
import numpy as np
from numpy.random import RandomState, SeedSequence, MT19937
from pathlib import Path

if '..' not in sys.path:
    sys.path = ['..'] + sys.path
from pfcommon import OU, combine_output_spectra
from filter_OU_inputs import run_welch

FIGURES_DIR = Path('figures')

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
fontsize = 9
lw = 0.75
matplotlib.rc('font', **{'family': 'Arial', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size': 3})
matplotlib.rc('ytick.major', **{'width': lw, 'size': 3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size': 1.5})

In [ ]:
use_dBs = True
dB = 20
dP = 0.01
assert dP >= 0.01
suffix = '_{:.02f}'.format(dP)

network_name = 'IEEE39_stoch'
network_name = 'SMs_with_line_and_loads'
network_name = 'SM_with_load'
if network_name == 'IEEE39_stoch':
    stoch_loads = ['Load_03']
    stoch_load_buses = [ld.replace('Load', 'Bus') for ld in stoch_loads]
    short_sim = True
    load_type = 'general_load'
elif network_name == 'SMs_with_line_and_loads':
    stoch_loads = ['LD1']
    stoch_load_buses = [ld.replace('LD', 'Bus') for ld in stoch_loads]
    short_sim = False
    load_type = 'general_load'
elif network_name == 'SM_with_load':
    stoch_loads = ['LD1']
    stoch_load_buses = [ld.replace('LD', 'Bus') for ld in stoch_loads]
    short_sim = False
    load_type = 'dynamic_load_const_S'

The folder where data are stored:

In [ ]:
base_data_dir = Path('../data')
data_dir = base_data_dir / network_name / load_type / '_'.join(stoch_loads)
assert os.path.isdir(data_dir)
tran_file = '{}_tran{}{}.npz'.format(network_name, suffix, '_short' if short_sim else '')
assert os.path.isfile(data_dir / tran_file)

Load the data from the transient simulation performed by PowerFactory:

In [ ]:
tran_data = np.load(data_dir / tran_file, allow_pickle=True)
data = tran_data['data'].item()

Make sure that the generators' speeds are constant around 1 p.u. (i.e., the network is stable):

In [ ]:
t_tran = tran_data['time']
try:
    gen_speed = data['gen']['s:speed']
except:
    gen_speed = data['gen']['s:xspeed']
fig,ax = plt.subplots(1, 1, figsize=(5,3))
ax.plot(t_tran, gen_speed, lw=0.75)
ax.set_xlabel('Time [s]')
ax.set_ylabel(r'$\omega_{gen}$ [p.u.]')
sns.despine()
fig.tight_layout()

#### Loads checks

Here I perform the following checks:
1. the generated active power of the load should match the one recorded from PF.
2. the PSD of both time series should match the theoretical one.

The PSD of an OU process is given by
$$
\mathrm{PSD}(f) = \frac{(\frac{c}{\alpha})^2}{1 + (\frac{2\pi f}{\alpha})^2},
$$
where $f$ is the frequency, $\alpha=1/\tau$, with $\tau$ the autocorrelation time constant of the process and $c=\sigma \sqrt{2/\tau}$, with $\sigma$ the (steady-state) standard deviation of the process.

In [ ]:
assert stoch_loads == tran_data['stoch_load_names'].tolist()
P0 = tran_data['stoch_load_P']
Q0 = tran_data['stoch_load_Q']
OU_seeds = tran_data['OU_seeds']
print('{:10s} {:>6s} {:>8s} {:>8s}'.format('LOAD NAME', 'P [MW]', 'Q [MVAR]', 'SEED'))
print('=' * 35)
for ld,p,q,seed in zip(stoch_loads, P0, Q0, OU_seeds):
    print('{:10s} {:6.2f} {:8.2f} {:8d}'.format(ld, p, q, seed))

In [ ]:
config = tran_data['config'].item()
dt_tran = config['dt']
N_samples = int(np.ceil(config['tstop'] / dt_tran)) + 1
t_OU = np.arange(N_samples) * dt_tran
P_OU_from_file = tran_data['OU_P']

τ = config['tau']['P']        # time constant
α = 1 / τ
P_OU = []
for i, (μ, seed) in enumerate(zip(P0, OU_seeds)):
    σ = config['sigma']['P'] * μ  # standard deviation
    c = σ * np.sqrt(2 / τ)
    rs = RandomState(MT19937(SeedSequence(seed)))
    ou = OU(dt_tran, μ, σ, τ, N_samples, rs)
    assert np.allclose(ou, P_OU_from_file[:,i]), 'Data from file does not match the generated OU time series'
    P_OU.append(ou)
P_OU = np.array(P_OU)

In [ ]:
U, phiu = [], []
I, phii = [], []
P_from_PF, Q_from_PF = [], []
device_names = tran_data['device_names'].item()
for ld, bus in zip(stoch_loads, stoch_load_buses):
    bus_idx = device_names['bus'].index(bus)
    ld_idx = device_names['load'].index(ld)
    try:
        U.append(data['bus']['m:U'][:, bus_idx])
        phiu.append(np.deg2rad(data['bus']['m:phiu'][:, bus_idx]))
        I.append(data['load']['m:I:bus1'][:, ld_idx])
        phii.append(np.deg2rad(data['load']['m:phii:bus1'][:, ld_idx]))
        P_from_PF.append(data['load']['m:Psum:bus1'][:, ld_idx])
        Q_from_PF.append(data['load']['m:Qsum:bus1'][:, ld_idx])
    except:
        assert len(stoch_loads) == 1 and bus_idx == 0 and ld_idx == 0
        U, phiu = data['bus']['m:U'], np.deg2rad(data['bus']['m:phiu'])
        I, phii = data['load']['m:I:bus1'], np.deg2rad(data['load']['m:phii:bus1'])
        P_from_PF = data['load']['m:Psum:bus1']
        Q_from_PF = data['load']['m:Qsum:bus1']
        break
U, phiu = np.array(U, ndmin=2), np.array(phiu, ndmin=2)
I, phii = np.array(I, ndmin=2), np.array(phii, ndmin=2)
P_from_PF, Q_from_PF = np.array(P_from_PF, ndmin=2), np.array(Q_from_PF, ndmin=2)

u = U * np.exp(1j * phiu)
i = I * np.exp(1j * phii)
S = 3 * u * i.conjugate()
P_from_PF_2 = S.real
Q_from_PF_2 = S.imag
assert np.allclose(P_from_PF, P_from_PF_2) and np.allclose(Q_from_PF, Q_from_PF_2)
Z = u / i

In [ ]:
def plot_loads(mean, stddev, tau, t_pf, P_pf, P_pf_2, Z_abs, t_ou, P_ou, window, Fmin, Fmax, onesided, ax):
    freq = np.logspace(Fmin, Fmax, 50*(Fmax-Fmin+1))
    alpha = 1 / tau
    c = stddev * np.sqrt(2 / tau)
    PSD_theor = (c / alpha) ** 2 / (1 + (2 * np.pi * freq / alpha) ** 2)
    freq_ou, PSD_ou, _ = run_welch(P_ou - mean, t_ou[1] - t_ou[0], window=window, onesided=onesided)
    freq_pf, PSD_pf, _ = run_welch(P_pf - P_pf.mean(), t_pf[1] - t_pf[0], window=window, onesided=onesided)

    tstop = 5
    jdx = t_pf <= tstop
    ax[0].plot([0, tstop], mean + np.zeros(2), 'k', lw=2, label='Pmean')
    ax[0].plot(t_pf[jdx], P_pf[jdx], color='tab:green', lw=0.75, label='From PF')
    ax[0].plot(t_pf[jdx], P_pf_2[jdx], '--', color='tab:red', lw=0.75, label='From PF - 2')
    ax[1].plot(t_pf[jdx], Z_abs[jdx], 'k', lw=0.75)
    jdx = t_ou <= tstop
    ax[0].plot(t_ou[jdx], P_ou[jdx], color='m', lw=1, label='OU computed from seed')
    ax[2].plot(freq_pf, PSD_pf, color='tab:green', lw=0.5, label='From PF')
    ax[2].plot(freq_ou, PSD_ou, color='m', lw=0.5, label='From file')
    ax[2].plot(freq, PSD_theor, 'k', lw=2, label='Theory')


N_window = 50 if short_sim else 200
N_loads = len(stoch_loads)
Fmin, Fmax, onesided = -6, 2, True
fig,ax = plt.subplots(3, N_loads, figsize=(3 * N_loads, 5.5), squeeze=False)
for i, (μ, p_ou, p_pf, p_pf_2, z) in enumerate(zip(P0, P_OU, P_from_PF, P_from_PF_2, Z)):
    σ = config['sigma']['P'] * μ
    plot_loads(μ, σ, τ, t_tran, p_pf, p_pf_2, np.abs(z), t_OU, p_ou, N_window/dt_tran, Fmin, Fmax, onesided, ax[:,i])

for i in range(N_loads):
    ax[0,i].set_xlabel('Time [s]')
    ax[1,i].set_xlabel('Time [s]')
    ax[2,i].set_xlabel('Frequency [Hz]')
    ax[2,i].set_xscale('log')
    ax[2,i].set_xlim([1e-2, 10**Fmax])
ax[0,0].set_ylabel('P [MW]')
ax[1,0].set_ylabel('|Z| [Ω]')
ax[2,0].set_ylabel(r'|Y(j$\omega$)|')
ax[0,0].legend(loc='best', frameon=True, fontsize=fontsize-2)
sns.despine()
fig.tight_layout(pad=0)
outfile = 'loads_and_spectra_{}_{}_{}{}.pdf'.format(network_name, load_type, '_'.join(stoch_loads), suffix)
print(f"Saving to file '{outfile}'")
plt.savefig(FIGURES_DIR / outfile)

Compute the spectra of the data obtained from the transient simulation:

In [ ]:
var_group_tran = 'bus'
if var_group_tran == 'gen':
    if network_name == 'IEEE39_stoch':
        device_name_tran = 'G_01'
    elif network_name == 'SMs_with_line_and_loads':
        device_name_tran = 'G2'
    elif network_name == 'SM_with_load':
        device_name_tran = 'G1'
    var_type_tran = 's:speed'
    # var_type_tran = 's:phi'
elif var_group_tran == 'bus':
    device_name_tran = stoch_load_buses[0]
    var_type_tran = 'm:ur'
    var_type_tran = 'm:ui'
    var_type_tran = 'm:u'
    var_type_tran = 'm:fe'
elif var_group_tran == 'load':
    device_name_tran = stoch_loads[0]
    var_type_tran = 's:xu'
    var_type_tran = 'm:ir:bus1'
    # var_type_tran = 'm:ii:bus1'
idx = device_names[var_group_tran].index(device_name_tran)
try:
    x_tran = data[var_group_tran][var_type_tran][:,idx]
except:
    assert idx == 0
    x_tran = data[var_group_tran][var_type_tran]
if var_type_tran == 'm:u':
    try:
        ur = data[var_group_tran]['m:ur'][:,idx]
        ui = data[var_group_tran]['m:ui'][:,idx]
    except:
        ur = data[var_group_tran]['m:ur']
        ui = data[var_group_tran]['m:ui']
    x_tran_check = np.sqrt(ur**2 + ui**2)
    assert np.allclose(x_tran, x_tran_check)
jdx = t_tran > 100
x_tran = x_tran[jdx]
t_tran = t_tran[jdx]
Δx_tran = x_tran - x_tran.mean()
freq_tran,P_tran,abs_tran = run_welch(Δx_tran, dt_tran, window=N_window/dt_tran, onesided=onesided)

if use_dBs:
    abs_tran = dB * np.log10(abs_tran)
    ylbl = r'|Y(j$\omega$)| [dB{}]'.format(dB)
else:
    ylbl = r'|Y(j$\omega$)|'

Load additional information about the transfer functions:

In [ ]:
steps_per_decade = 100 if 'IEEE39' in network_name else 1000
TF_file = '{}_TF_-6.0_2.0_{}_{:.2f}.npz'.format(network_name, steps_per_decade, dP)
assert os.path.isfile(data_dir / TF_file)
TF_data = np.load(data_dir / TF_file, allow_pickle=True)
assert stoch_loads == TF_data['load_names'].tolist()
PF = TF_data['PF'].item()
# these are the loads for which individual TFs were computed by compute_spectra.py:
# they are NOT necessarily all the loads that are present in the power network
all_load_names = TF_data['load_names'].tolist()
all_var_names = TF_data['var_names'].tolist()
assert all([ld in all_load_names for ld in stoch_loads])

Load the data from the small-signal simulation, i.e., the variables obtained by filtering the input(s) with the appropriate transfer functions:

In [ ]:
small_signal_file = '{}_ss{}.h5'.format(network_name, suffix)
fid = h5py.File(data_dir / small_signal_file)
t_ss = np.array(fid['time'])
def read_list(fid, key):
    names = fid['parameters'][key].tolist()[0]
    return list(map(lambda n: n.decode('utf-8'), names))
load_names = read_list(fid, 'load_names')
assert load_names == stoch_loads
var_names_ss = read_list(fid, 'var_names')

var_type_ss = var_type_tran.split(':')[1]
if var_group_tran == 'bus':
    var_group_ss = 'Term'
elif var_group_tran == 'gen':
    var_group_ss = 'Sym'
elif var_group_tran == 'load':
    var_group_ss = 'Lod'
else:
    raise Exception(f"Unknown variable group '{var_group_tran}'")
var_name_ss = 'Grid-{}.Elm{}.{}'.format(device_name_tran, var_group_ss, var_type_ss)
if var_name_ss not in var_names_ss:
    var_names_ss.append(var_name_ss)
var_name_ss = var_name_ss.replace('-', '_').replace('.', '_')

try:
    x_ss = np.array(fid[var_name_ss]).squeeze()
    print(f"Variable '{var_name_ss}' present in data file.")
except:
    print(f"Variable '{var_name_ss}' not directly available in data file: composing it from other variables.")
    if var_type_ss == 'u':
        ur, ui = PF['buses'][device_name_tran]['ur'], PF['buses'][device_name_tran]['ui']
        coeff_ur, coeff_ui = np.array([ur, ui]) / np.sqrt(ur**2 + ui**2)
        x_ss = coeff_ur * np.array(fid[var_name_ss[:-1] + 'ur']).squeeze() + \
               coeff_ui * np.array(fid[var_name_ss[:-1] + 'ui']).squeeze()
    else:
        print(f"Do not know how to compose variable '{var_name_ss}'.")
        if 'phi' in var_name_ss:
            print("Are you sure you are not asking for the angle of the slack generator? That wouldn't make sense.")
fid.close()

Build the theoretical PSDs of the output variables:

In [ ]:
F0 = 50.
F = TF_data['F']
var_types = [os.path.splitext(name)[1][1:] for name in var_names_ss]
OUT_multi = combine_output_spectra(TF_data['OUT'], stoch_loads, var_names_ss, all_load_names,
                                   all_var_names, var_types, F, PF,
                                   TF_data['bus_equiv_terms'].item(), ref_freq=F0,
                                   ref_SM_name=TF_data['ref_SM_name'].item())
out = np.abs(OUT_multi)
if use_dBs:
    out = dB*np.log10(out)

Compute the spectra of the data obtained from the small signal simulation:

In [ ]:
dt_ss = t_ss[1] - t_ss[0]
Δx_ss = x_ss - x_ss.mean()
freq_ss,P_ss,abs_ss = run_welch(Δx_ss, dt_ss, window=N_window/dt_ss, onesided=onesided)
if use_dBs:
    abs_ss = dB * np.log10(abs_ss)

In [ ]:
from scipy.integrate import trapezoid
idx = [n.replace('.','_').replace('-','_') for n in var_names_ss].index(var_name_ss)
print('\n===== Small signal data =====')
print('Integral of the theoretical PSD: {:g}'.format(trapezoid(out[idx]**2 * (1 + onesided), F)))
print('Integral of the numerical PSD: {:g}'.format(trapezoid(P_ss * (1 + onesided), freq_ss)))
print('Variance of the signal: {:g}'.format(np.var(x_ss)))
print('\n===== PF transient data =====')
print('Integral of the PSD: {:g}'.format(trapezoid(P_tran * (1 + onesided), freq_tran)))
print('Variance of the signal: {:g}'.format(np.var(x_tran)))

In [ ]:
typ = var_name_ss.split('_')[-1]
fig,ax = plt.subplots(2, 1, figsize=(5,5))

remove_mean = True
if remove_mean:
    ax[0].plot(t_tran, Δx_tran, 'k', lw=0.75, label='Tran')
    ax[0].plot(t_ss, Δx_ss, 'tab:green', lw=0.75, label='S.S.', alpha=0.75)
    ax[0].set_ylabel(f'Δ{typ} [p.u.]')
else:
    twin_ax = ax[0].twinx()
    ax[0].plot(t_tran, x_tran, 'k', lw=0.75, label='Tran')
    twin_ax.plot(t_ss, x_ss, 'tab:green', lw=0.75, label='S.S.', alpha=0.75)
    ax[0].set_ylabel(r'${}_t$ [p.u.]'.format(typ))
    twin_ax.set_ylabel(r'${}_s$ [p.u.]'.format(typ))
ax[0].set_xlabel('Time [s]')

ax[1].plot(freq_tran, abs_tran, 'k', lw=0.75, label=r'Tran ($\sigma^2$ = {:.1e})'.\
               format(trapezoid(P_tran * (1 + onesided), freq_tran)))
ax[1].plot(freq_ss, abs_ss, 'tab:green', lw=0.75, label='S.S. ($\sigma^2$ = {:.1e})'.\
               format(trapezoid(P_ss * (1 + onesided), freq_ss)), alpha=0.75)
ax[1].plot(F, out[idx], 'tab:red', lw=2, label='Theory')
loc = 'lower left' if use_dBs else 'best'
ax[1].legend(loc=loc, frameon=False, fontsize=fontsize-1)
ax[1].set_xscale('log')
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_ylabel(ylbl)
ax[1].set_xlim([1e-3, F[-1]])
# ax[1].set_xlim([1, 5])
if remove_mean:
    sns.despine()
fig.tight_layout()
outfile = 'spectra_comparison_tran_ss_{}_{}_{}{}_{}_{}.pdf'.format(network_name, load_type, '_'.join(stoch_loads),
                                                                   suffix, device_name_tran.split('__')[0], typ)
print(f"Saving to file '{outfile}'")
plt.savefig(FIGURES_DIR / outfile)